In [23]:
#Some of this code was based on code acquired through ChatGPT prompts
#To begin, we will import our packages
import arcpy
import os
import psycopg2
from psycopg2 import sql

In [24]:
# First I establish the coordinates of the vertices (lon, lat)
vertex1 = (-74.0, 40.0)  # Lower left corner
vertex2 = (-74.0, 41.0)  # Upper left corner
vertex3 = (-73.0, 41.0)  # Upper right corner

# Then I can create a new polygon geometry object
polygon = arcpy.Polygon(arcpy.Array([arcpy.Point(*vertex) for vertex in [vertex1, vertex2, vertex3]]))

# I will convert the polygon into well-known text and print it
wkt_polygon = polygon.WKT
print(wkt_polygon)

MULTIPOLYGON (((-74 40.0001220703125, -73 41.0001220703125, -74 41.0001220703125, -74 40.0001220703125)))


In [26]:
# With the polygon ready, I then need to establish a connection to the PostgreSQL database using psycopg2
conn = psycopg2.connect(database="DATABASE",
                        user="USER",
                        password="PASSWORD",
                        host="HOST",
                        port="PORT")

# I create a cursor object to execute SQL queries within the connection
cur = conn.cursor()

# I will then define an SQL query to insert a geometry into the labtable
insert_query = "INSERT INTO labtable (geom) VALUES (%s);"

# I set the table name
table_name = "labtable"

# I can create a table, specifying the table name, id column, and geometry column
create_table_query = sql.SQL("""
    CREATE TABLE IF NOT EXISTS {} (
        id SERIAL PRIMARY KEY,
        geom text
    )
""").format(sql.Identifier(table_name))

# I can then execute the create table query
cur.execute(create_table_query)

# Then I can execute the insert query, inserting a WKT polygon into the labtable
cur.execute(insert_query, (wkt_polygon,))

# I commit the transaction to save the changes to the database
conn.commit()

# I can close the cursor to release database resources and close the connection
cur.close()
conn.close()